<a href="https://colab.research.google.com/github/isabellacadisco/text-data-for-trading-/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.0 MB/s eta 0:00:00


In [ ]:
# To finetune transformer model, we need following libraries
import tensorflow as tf
from tensorflow.keras import activations, optimizers, losses
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification

# To save our trained model, we import pickle library
import pickle

# To pre-process, we need following libraries
import pandas as pd
import numpy as np
import seaborn as sns

# To explre and visualize data, we need following libraries
import matplotlib.pyplot as plt
import plotly.express as px

# To read the data from google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datetime import date
from scipy import stats
import statsmodels.api as sm
import plotly.express as px
from plotly import graph_objects as go
import matplotlib.pyplot as plt
import scipy
import time

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Ammagamma Lab Project/news_archive.csv')

In [ ]:
df

,txt,url,date
0,\nCrude Oil - Brent Oil: Break below $70 could...,http://www.forexrate.co.uk/Crude-Oil---Brent-O...,13-06-2023 02:41
1,\nSilver Price - Silver Price Analysis: XAG/US...,http://www.forexrate.co.uk/Silver-Price---Silv...,13-06-2023 01:25
2,\nOil Price - USD/CAD hangs near one-month low...,http://www.forexrate.co.uk/Oil-Price---USD-CAD...,12-06-2023 02:04
3,\nUSDJPY - USD/JPY faces extra range bound nea...,http://www.forexrate.co.uk/USDJPY---USD-JPY-fa...,12-06-2023 00:30
4,\nGold Price - Gold bound for weekly gain on F...,http://www.forexrate.co.uk/Gold-Price---Gold-b...,09-06-2023 03:16
...,...,...,...
2677,\nBitcoin dives double-digits as South Korea p...,http://www.forexrate.co.uk/Bitcoin-dives-doubl...,11-01-2018 08:04
2678,\nCommodities: WTI futures gain after DoE inve...,http://www.forexrate.co.uk/Commodities--WTI-fu...,04-01-2018 18:14
2679,\nCommodities: Energy futures jump on US inven...,http://www.forexrate.co.uk/Commodities--Energy...,03-01-2018 17:55
2680,"\nFX round-up: Pound jumps, JP Morgan sees ""pe...",http://www.forexrate.co.uk/FX-round-up--Pound-...,02-01-2018 20:41


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2682 entries, 0 to 2681
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   txt     2682 non-null   object
 1   url     2682 non-null   object
 2   date    2682 non-null   object
dtypes: object(3)
memory usage: 63.0+ KB


**Aggiungere filtro related US EU economies or not**

**Classificatore applicato a news**

In [ ]:
new_model = TFDistilBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/Ammagamma Lab Project/model/clf')
model_name, max_len = pickle.load(open('/content/drive/MyDrive/Ammagamma Lab Project/model/info.pkl', 'rb'))

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/Ammagamma Lab Project/model/clf.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [ ]:
# metto qui funzione create_predictor ma sarebbe bello avere tutte le funzioni in file a parte

def create_predictor(model, model_name, max_len):
  tokenizer = DistilBertTokenizer.from_pretrained(model_name)
  def predict_proba(text):
      x = [text]

      encodings = construct_encodings(x, tokenizer, max_len=max_len)
      tfdataset = construct_tfdataset(encodings)
      tfdataset = tfdataset.batch(1)

      preds = model.predict(tfdataset).logits
      #print("pred logits: ",preds)
      preds = activations.softmax(tf.convert_to_tensor(preds)).numpy()
      #print("softmax o/p: ",preds)
      return preds[0][0], preds[0][1], preds[0][2]

  return predict_proba


def construct_encodings(x, tokenizer, max_len, trucation=True, padding=True):
  return tokenizer(x, max_length=max_len, truncation=trucation, padding=padding)

#encodings = construct_encodings(x, tokenizer, max_len=MAX_LEN)


def construct_tfdataset(encodings, y=None):
    if y:
        return tf.data.Dataset.from_tensor_slices((dict(encodings),y))
    else:
        # this case is used when making predictions on unseen samples after training
        return tf.data.Dataset.from_tensor_slices(dict(encodings))

# tokenized data in the form of tensor slices



In [ ]:
clf = create_predictor(new_model, model_name, max_len)

In [ ]:
# Apply classifier to each text and store the results in new columns
df[['Negative', 'Neutral', 'Positive']] = pd.DataFrame(df['txt'].apply(clf).tolist())


1/1 [==============================] - 0s 190ms/step


In [ ]:
df

,txt,url,date,Negative,Neutral,Positive
0,\nCrude Oil - Brent Oil: Break below $70 could...,http://www.forexrate.co.uk/Crude-Oil---Brent-O...,13-06-2023 02:41,0.977927,0.002546,0.019527
1,\nSilver Price - Silver Price Analysis: XAG/US...,http://www.forexrate.co.uk/Silver-Price---Silv...,13-06-2023 01:25,0.081250,0.003776,0.914974
2,\nOil Price - USD/CAD hangs near one-month low...,http://www.forexrate.co.uk/Oil-Price---USD-CAD...,12-06-2023 02:04,0.977453,0.002681,0.019866
3,\nUSDJPY - USD/JPY faces extra range bound nea...,http://www.forexrate.co.uk/USDJPY---USD-JPY-fa...,12-06-2023 00:30,0.018682,0.003003,0.978315
4,\nGold Price - Gold bound for weekly gain on F...,http://www.forexrate.co.uk/Gold-Price---Gold-b...,09-06-2023 03:16,0.216322,0.004808,0.778870
...,...,...,...,...,...,...
2677,\nBitcoin dives double-digits as South Korea p...,http://www.forexrate.co.uk/Bitcoin-dives-doubl...,11-01-2018 08:04,0.980362,0.002982,0.016656
2678,\nCommodities: WTI futures gain after DoE inve...,http://www.forexrate.co.uk/Commodities--WTI-fu...,04-01-2018 18:14,0.928915,0.004065,0.067020
2679,\nCommodities: Energy futures jump on US inven...,http://www.forexrate.co.uk/Commodities--Energy...,03-01-2018 17:55,0.706264,0.004799,0.288938
2680,"\nFX round-up: Pound jumps, JP Morgan sees ""pe...",http://www.forexrate.co.uk/FX-round-up--Pound-...,02-01-2018 20:41,0.970062,0.002833,0.027105


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2682 entries, 0 to 2681
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   txt       2682 non-null   object 
 1   url       2682 non-null   object 
 2   date      2682 non-null   object 
 3   Negative  2682 non-null   float32
 4   Neutral   2682 non-null   float32
 5   Positive  2682 non-null   float32
dtypes: float32(3), object(3)
memory usage: 94.4+ KB


In [ ]:
print(df['date'].dtype)

object


In [ ]:
df['date'] = pd.to_datetime(df['date']).dt.date


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2682 entries, 0 to 2681
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   txt       2682 non-null   object 
 1   url       2682 non-null   object 
 2   date      2682 non-null   object 
 3   Negative  2682 non-null   float32
 4   Neutral   2682 non-null   float32
 5   Positive  2682 non-null   float32
dtypes: float32(3), object(3)
memory usage: 94.4+ KB


**Aggregate (x, y, z) on dates**

In [ ]:
df

,txt,url,date,Negative,Neutral,Positive
0,\nCrude Oil - Brent Oil: Break below $70 could...,http://www.forexrate.co.uk/Crude-Oil---Brent-O...,2023-06-13,0.977927,0.002546,0.019527
1,\nSilver Price - Silver Price Analysis: XAG/US...,http://www.forexrate.co.uk/Silver-Price---Silv...,2023-06-13,0.081250,0.003776,0.914974
2,\nOil Price - USD/CAD hangs near one-month low...,http://www.forexrate.co.uk/Oil-Price---USD-CAD...,2023-12-06,0.977453,0.002681,0.019866
3,\nUSDJPY - USD/JPY faces extra range bound nea...,http://www.forexrate.co.uk/USDJPY---USD-JPY-fa...,2023-12-06,0.018682,0.003003,0.978315
4,\nGold Price - Gold bound for weekly gain on F...,http://www.forexrate.co.uk/Gold-Price---Gold-b...,2023-09-06,0.216322,0.004808,0.778870
...,...,...,...,...,...,...
2677,\nBitcoin dives double-digits as South Korea p...,http://www.forexrate.co.uk/Bitcoin-dives-doubl...,2018-11-01,0.980362,0.002982,0.016656
2678,\nCommodities: WTI futures gain after DoE inve...,http://www.forexrate.co.uk/Commodities--WTI-fu...,2018-04-01,0.928915,0.004065,0.067020
2679,\nCommodities: Energy futures jump on US inven...,http://www.forexrate.co.uk/Commodities--Energy...,2018-03-01,0.706264,0.004799,0.288938
2680,"\nFX round-up: Pound jumps, JP Morgan sees ""pe...",http://www.forexrate.co.uk/FX-round-up--Pound-...,2018-02-01,0.970062,0.002833,0.027105


In [ ]:
# Group by date and calculate the mean of the probabilities
df_agg = df.groupby('date').mean().reset_index()


<ipython-input-28-87d43c1cb809>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_agg = df.groupby('date').mean().reset_index()


In [ ]:
df_agg

,date,Negative,Neutral,Positive
0,2018-01-02,0.982824,0.002747,0.014429
1,2018-01-03,0.174240,0.003935,0.821825
2,2018-01-05,0.955479,0.003668,0.040853
3,2018-01-06,0.985943,0.002905,0.011153
4,2018-01-08,0.502520,0.172853,0.324627
...,...,...,...,...
1348,2023-11-05,0.617450,0.004003,0.378547
1349,2023-12-01,0.970847,0.002861,0.026292
1350,2023-12-04,0.175564,0.491626,0.332810
1351,2023-12-05,0.508512,0.002792,0.488696


In [ ]:
df_agg.to_csv('/content/drive/MyDrive/Ammagamma Lab Project/sentiment_extra_reg.csv')

In [ ]:
extra_reg = pd.read_csv("/content/drive/MyDrive/Copia di extra_regressors.csv")

In [ ]:
extra_reg

,DATE,CPIAUCSL,GDP,T10YIE
0,1947-01-01,21.480,243.164,NaN
1,1947-01-02,21.480,243.164,NaN
2,1947-01-03,21.480,243.164,NaN
3,1947-01-04,21.480,243.164,NaN
4,1947-01-05,21.480,243.164,NaN
...,...,...,...,...
27164,2021-05-16,266.832,22048.894,2.51
27165,2021-05-17,266.832,22048.894,2.54
27166,2021-05-18,266.832,22048.894,2.52
27167,2021-05-19,266.832,22048.894,2.48
